<a href="https://colab.research.google.com/github/nogbazghi/CSC8980/blob/main/Exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nahom Ogbazghi

002052292

In [ ]:

!pip install wget
import wget

print('Downloading dataset...')
!unzip exam1_dataset.zip

In [6]:
import os
from sklearn.datasets import load_files

#I did not see a random seed but a random state
random_state = 12345

data = load_files('exam1_dataset/TRAINING', random_state=random_state)
categories = data.target_names

class Pairs:
  def __init__(self, data, target):
        self.data = data
        self.target = target

Question 1) (20 points) Write a generic function that takes: Classification algorithm name,
vectorization method name, training set with labels as parameters (total of 3 parameters should
be passed). The function should take the classification algorithm name, the vectorization
method’s name, and the training set and train the desired model. Use the default training
parameters for the models we have seen in class. This function should return the trained model.


In [7]:
from sklearn.pipeline import make_pipeline

def trainModel(classificationAlgorithmName, vectorizationMethodName, train):
  model = make_pipeline(vectorizationMethodName, classificationAlgorithmName)
  model.fit(train.data, train.target)
  return model


Question 2) (30 points) Using the function from question 1 to build the following models:
a) Model a: Naive Bayes, Vectorizer: TFIDF and Bag of Words, Training set should be 75%
of the provided dataset. Leaving the remaining 25% for testing.
b) Model b: RandomForest, Vectorizer: TFIDF and Bag of Words, Training set should be
70% of the provided dataset. Leaving the remaining 30% for testing.
c) Model c: Support Vector Machines (SVC in sklearn), Vectorizer: TFIDF and Bag of
Words, Training set should be 60% of the provided dataset. Leaving the remaining 40%
for testing.
NOTE: Set the random seed to: 12345. This needs to be consistently set to train the model
AND split the data in test and train. If this is not done correctly, you will lose points as your
answers will not be comparable with the grading key.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

vTFIDF = TfidfVectorizer()
vBOW = CountVectorizer()
data_to_split = data.data
target_to_split = data.target

#I did not see a random seed but a random state
random_state = 12345

print("Naive Bayes Training")
nb_training_data, nb_testing_data, nb_training_target, nb_testing_target = train_test_split(data_to_split, target_to_split, train_size=0.75)
nb_training = Pairs(nb_training_data, nb_training_target)
print("TFIDF")
nb_tf_model = trainModel(MultinomialNB(), TfidfVectorizer(), nb_training)
nb_tf_model.fit(nb_training.data, nb_training.target)
print("BOW")
nb_bow_model = trainModel(MultinomialNB(), CountVectorizer(), nb_training)
nb_bow_model.fit(nb_training.data, nb_training.target)

print("Random Forest Training")
rf_training_data, rf_testing_data, rf_training_target, rf_testing_target = train_test_split(data_to_split, target_to_split, train_size=0.7)
rf_training = Pairs(rf_training_data, rf_training_target)
print("TFIDF")
rf_tf_model = trainModel(RandomForestClassifier(random_state=random_state), TfidfVectorizer(), rf_training)
rf_tf_model.fit(rf_training.data, rf_training.target)
print("BOW")
rf_bow_model = trainModel(RandomForestClassifier(random_state=random_state), CountVectorizer(), rf_training)
rf_bow_model.fit(rf_training.data, rf_training.target)

print("SVM Training")
#I tried using SVC, but it repeatedly crashed my runs so I opted to use LinearSVC
svm_training_data, svm_testing_data, svm_training_target, svm_testing_target = train_test_split(data_to_split, target_to_split, train_size=0.6, random_state=random_state)
svm_training = Pairs(svm_training_data, svm_training_target)
print("TFIDF")
svm_tf_model = trainModel(LinearSVC(random_state=random_state), TfidfVectorizer(), svm_training)
svm_tf_model.fit(svm_training.data, svm_training.target)
print("BOW")
svm_bow_model = trainModel(LinearSVC(random_state=random_state), CountVectorizer(), svm_training)
svm_bow_model.fit(svm_training.data, svm_training.target)


Naive Bayes Training
TFIDF
BOW
Random Forest Training
TFIDF
BOW
SVM Training
TFIDF
BOW


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=12345,
                  

Question 3) (30 points) Using the models from Question 2, evaluate each model with its
respective training set (for model a, that set is 25% of the data, for model b, that set is 30% of
the data, and for model c that set is 40% of the data. Be careful to not mix up the evaluation
sets. With the predictions on the test set and show the following metrics: Accuracy, Precision,
Recall, and Macro F1-score. With this in mind, please write and answer these questions in your
notebook:
a) What model performs the best and why? (which metrics do you base this on, and why do
you think it performs better than others).
b) Why is it important not to mix up the testing sets between different models? Think about
this one.
c) Display in a single sorted dataframe (model name, training %, test %, accuracy,
precision, recall, F1-score) all performance metrics, sorted by accuracy in descending
manner

In [9]:
import sklearn.metrics
import pandas as pd
import numpy as np

print("Naive Baies Prediction")
print("TFIDF")
nb_tf_labels = nb_tf_model.predict(nb_testing_data)
nb_tfList = ["NB_TFIDF", 0.75, 0.25, sklearn.metrics.accuracy_score(nb_testing_target, nb_tf_labels), sklearn.metrics.precision_score(nb_testing_target, nb_tf_labels), sklearn.metrics.recall_score(nb_testing_target, nb_tf_labels), sklearn.metrics.f1_score(nb_tf_labels, nb_testing_target, average='macro')]
print("BOW")
nb_bow_labels = nb_bow_model.predict(nb_testing_data)
nb_bowList = ["NB_BOW", 0.75, 0.25, sklearn.metrics.accuracy_score(nb_testing_target, nb_bow_labels), sklearn.metrics.precision_score(nb_testing_target, nb_bow_labels), sklearn.metrics.recall_score(nb_testing_target, nb_bow_labels), sklearn.metrics.f1_score(nb_bow_labels, nb_testing_target, average='macro')]

print("Random Forest Predicting")
print("TFIDF")
rf_tf_labels = rf_tf_model.predict(rf_testing_data)
rf_tfList = ["RF_TFIDF", 0.7, 0.3, sklearn.metrics.accuracy_score(rf_testing_target, rf_tf_labels), sklearn.metrics.precision_score(rf_testing_target, rf_tf_labels), sklearn.metrics.recall_score(rf_testing_target, rf_tf_labels), sklearn.metrics.f1_score(rf_tf_labels, rf_testing_target, average='macro')]
print("BOW")
rf_bow_labels = rf_bow_model.predict(rf_testing_data)
rf_bowList = ["RF_BOW", 0.7, 0.3, sklearn.metrics.accuracy_score(rf_testing_target, rf_bow_labels), sklearn.metrics.precision_score(rf_testing_target, rf_bow_labels), sklearn.metrics.recall_score(rf_testing_target, rf_bow_labels), sklearn.metrics.f1_score(rf_bow_labels, rf_testing_target, average='macro')]

print("SVM Prediction")
print("TFIDF")
svm_tf_labels = svm_tf_model.predict(svm_testing_data)
svm_tfList = ["SVM_TFIDF", 0.6, 0.4, sklearn.metrics.accuracy_score(svm_testing_target, svm_tf_labels), sklearn.metrics.precision_score(svm_testing_target, svm_tf_labels), sklearn.metrics.recall_score(svm_testing_target, svm_tf_labels), sklearn.metrics.f1_score(svm_tf_labels, svm_testing_target, average='macro')]
print("BOW")
svm_bow_labels = svm_bow_model.predict(svm_testing_data)
svm_bowList = ["SVM_BOW", 0.6, 0.4, sklearn.metrics.accuracy_score(svm_testing_target, svm_bow_labels), sklearn.metrics.precision_score(svm_testing_target, svm_bow_labels), sklearn.metrics.recall_score(svm_testing_target, svm_bow_labels), sklearn.metrics.f1_score(svm_bow_labels, svm_testing_target, average='macro')]


Naive Baies Prediction
TFIDF
BOW
Random Forest Predicting
TFIDF
BOW
SVM Prediction
TFIDF
BOW


a) What model performs the best and why? (which metrics do you base this on, and why do you think it 
performs better than others). 

SVM using TFIDF performed the best. I base this primarily on Recall since it scored the highest in that and it can be double checked for any false positives as it is easy to do so with high recalls. It performed well because it is good for accuracy, but it may overfit.

b) Why is it important not to mix up the testing sets between different models? Think about this one. 

It's important not to mix the testing sets between different models because the splits allocate a percentage of the data between a test and train sets and if a model that was originally trained with NB training set(75%), but was then tested with SVMs testing set(40%) there will be an overlap of testing data that is within the trained model. This will return a higher than normal score since the model will essentially be tested on data that it already knows. 

c) Display in a single sorted dataframe (model name, training %, test %, accuracy, precision, recall, F1-score) all performance metrics, sorted by accuracy in descending manner

In [10]:
listofResults = [nb_tfList, nb_bowList, rf_tfList, rf_bowList, svm_tfList, svm_bowList]
df = pd.DataFrame(np.array(listofResults), columns=["Name", "Training %", "Test %", "Accuracy", "Precision", "Recall", "F1 Score"])
sortedDf = df.sort_values(by="Accuracy", ascending=False)
print(sortedDf)

        Name Training %  ...              Recall            F1 Score
4  SVM_TFIDF        0.6  ...  0.8926936093967749  0.8894941111411827
5    SVM_BOW        0.6  ...  0.8582520406131794   0.854591601210886
0   NB_TFIDF       0.75  ...  0.8354964308890331  0.8518838769595363
1     NB_BOW       0.75  ...  0.8072680077871512  0.8377949592879452
3     RF_BOW        0.7  ...  0.8557227297152069  0.8373099059597915
2   RF_TFIDF        0.7  ...  0.8288554540569586  0.8275958040649913

[6 rows x 7 columns]


Question 4) (15 points) Using the documents in the folder named UNLABELED, please use
your best performing trained model from question 3 to predict their labels. Please do this
individually for each document. Print to the screen the following items: Document Name,
Predicted Label and using a text cell, write your own opinion if the label is correct and why -
note you have to read the document to make your own opinion.

**

*   filename: 46278_0.txt | predicted labeled: positive | yes, they were originally look warm but at the end it was positive 
*   filename: 36517_0.txt | predicted labeled: negative | yes, they described it as drespressingly dreary
*   filename: 24221_0.txt | predicted labeled: negative | they could not recommend it 
*   filename: 46705_0.txt | predicted labeled: negative | yes, they mentioned it only got worse and worse
*   filename: 37154_0.txt | predicted labeled: negative |  yes, they listed several negatives
*   filename: 0_0.txt | predicted labeled: positive | I think it read negative, but labeled positive, so not accurate
*   filename: 36022_0.txt | predicted labeled: negative | yes, they mentioned the creator had better films
*   filename: 36149_0.txt | predicted labeled: positive | no, the user described it as making them sick 
*   filename: 35991_0.txt | predicted labeled: negative | yes, they said it was a bad film
*   filename: 49990_0.txt | predicted labeled: negative | yes, used words like sadly nonexistant
*   filename: 35968_0.txt | predicted labeled: positive | no, I think they described overall negatively





**

In [11]:
path = '/content/exam1_dataset/UNLABELED/'
for filename in os.listdir(path):
  if filename.endswith(".txt"):
    file = open(path+str(filename), 'r')
    content = file.read()
    label = svm_tf_model.predict([content])
    print("filename:",filename, "| predicted labeled:", categories[label[0]])


['negative', 'positive']
filename: 46278_0.txt | predicted labeled: positive
filename: 36517_0.txt | predicted labeled: negative
filename: 24221_0.txt | predicted labeled: negative
filename: 46705_0.txt | predicted labeled: negative
filename: 37154_0.txt | predicted labeled: negative
filename: 0_0.txt | predicted labeled: positive
filename: 36022_0.txt | predicted labeled: negative
filename: 36149_0.txt | predicted labeled: positive
filename: 35991_0.txt | predicted labeled: negative
filename: 49990_0.txt | predicted labeled: negative
filename: 35968_0.txt | predicted labeled: positive


Question 5) (20 points) Build a function that takes the set of documents as input and returns a
cosine similarity matrix for those documents. Feed all documents in the TRAINING folder to this
matrix. Instead of printing the returned cosine similarity matrix, create a heatmap plot from the
returned matrix. Make sure your plot is nicely scaled, properly labeled, and uses a nice color
range to show the similarity.


In [17]:
import nltk

nltk.download('reuters')
nltk.download('punkt')
allFilenames = []
documents = []
path = '/content/exam1_dataset/TRAINING/'
for c in categories:
  for f in os.listdir(path+c+"/"):
    filename = c+"/"+str(f)
    allFilenames.append(str(filename))
    file = open(path+str(filename), 'r')
    content = file.read()
    documents.append((filename, content))

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
import math
import sklearn.metrics
from nltk.tokenize import word_tokenize

def getMatrix(documents):
  matrix = {}
  for doc in documents:
    filename = doc[0]
    content = doc[1]
    content = file.read()
    tokens = word_tokenize(content)
    for word in tokens:
      if word in matrix:
        wordFilenameCount = matrix[word]
        if filename in wordFilenameCount: 
          wCcount = wordFilenameCount[filename]
          wordFilenameCount[filename] = wCcount + 1
          matrix[word] = wordFilenameCount
        else:
          wordFilenameCount[filename] = 1
          matrix[word] = wordFilenameCount
      else:
        insertDictIntoMW={}
        insertDictIntoMW[filename] = 1
        matrix[word] = insertDictIntoMW
  return matrix


def find_tf_id(count):
  return math.log(count+1, 10)

def find_df(matrix, word):
  docs = matrix[word]
  return len(docs.keys())

def find_idf(matrix, word):
  df = find_df(matrix, word)
  count = len(allFilenames)/df
  return math.log(count, 10)

def find_tf_idf(tf, idf):
  return tf * idf

# idf per word
def get_idf_matrix(words, matrix):
  imatrix = {}
  for word in words:
    idf = find_idf(matrix, word)
    imatrix[word] = idf
  return imatrix

# tf idf per word
def get_tfidf_matrid(matrix, words, idf_matrix):
  tmatrix = {} #key:filename, value: {key:word, value:tf idf} 
  for word in words:
    wcPerFileForWord = matrix[word]
    filenamesOfWord = wcPerFileForWord.keys()
    # print(filenamesOfWord)
    for filename in filenamesOfWord:
      wordCount = wcPerFileForWord[filename]
      tf = find_tf_id(wordCount)
      idf = idf_matrix[word]
      if filename in tmatrix:
        tfsPerWord = tmatrix[filename]
      else:
        tfsPerWord = {}
      tfsPerWord[word] = find_tf_idf(tf, idf)
    tmatrix[filename] = tfsPerWord
  return tmatrix

def cosSimOfTwoDocs(fileA, fileB, tf_idf_matrix):
  fileAwks = list(tf_idf_matrix[fileA].values())
  fileBwks = list(tf_idf_matrix[fileB].values())
  kA = len(fileAwks)
  kB = len(fileBwks)
  for tfidfScore in fileAwks:
    anum = anum + tfidfScore
  
  for tfidfScore in fileBwks:
    bnum = bnum + tfidfScore
  dA = anum/kB
  dB = bnum/kB
  num = dA + dB
  den = dA * dB
  return num/den


def getcosDocPairs(documents, tfidfMatrix):
  cosDocPairs = {}
  for fileA,content in documents:
    cosSimVals = {}
    filesinCosDocPairs = cosDocPairs.keys()
    if fileA in filesinCosDocPairs:
      cosSimVals = cosDocPairs[fileA]
    for fileB,content in documents:
      cosSim = cosSimOfTwoDocs(fileA, fileB, tfidfMatrix)
      cosSimVals[fileB] = cosSim
    cosDocPairs[fileA] = cosSimVals
  return cosDocPairs

matrix = getMatrix(allFilenames)
words = matrix.keys()
idf_matrix = get_idf_matrix(words, matrix)
tf_idf_matrix = get_tfidf_matrid(matrix, words, idf_matrix)
cosDocPairs = getcosDocPairs(documents, tf_idf_matrix)

KeyError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix


Question 6) (15 points) Write a function that takes a cosine similarity matrix as input and
returns a list with the top n document paris and their similarity. Note that you should only keep
the document pairs that are unique and remove the comparisons of the document to itself. Print
the top 50 similar document pairs. Compare the assigned class for each document and answer:
Do all similar documents belong to the same class? Why or why not?

**
All assigned classes did belong to the same class, because they used similar words to describe the film. positive reviews used similar positive words, while negative reviews used similar negative words **

In [ ]:
def replacedSmallest(m, num, nameA, nameB):
  values = list(m.values())
  values.sort()
  smallest = values[0]
  if(num > smallest):
    names = m.keys()
    for n in names:
      if m[n] == smallest:
        m.pop(n)
        m[(nameA,nameB)] = num
  return m

def alreadyInList(m, nameA, nameB):
  names = m.keys()
  for n in names:
    if nameA in n:
      if nameB in n:
        return True
  return False

def topNSimilarWorks(cosDocPairs):
  n = 50
  nWorks = {}
  for fA in allFilenames:
    cosList = cosDocPairs[fA]
    for fB in allFilenames:
      if alreadyInList(nWorks, fA, fB) == False: #already seen
        if fA != fB: # same pair
          value = cosList[fB]
          if len(nWorks) == n:
            nWorks = replacedSmallest(nWorks, value, fA, fB)
          else:
            nWorks[(fA,fB)] = value
  return nWorks.keys()

items = topNSimilarWorks(50)
print("Top", len(items), "similar files")
for item in items:
  first = cosDocPairs[item[0]]
  print ((item[0], item[1]))

Question 7) (20 points) Using Spacy’s part of speech tagger, process all sentences (hint: don’t
forget to split the reviews) and count how many NOUN and VERB tags are found in all the
movies review (TRAINING folder) separating them by label. In other words, how many NOUN
and VERB tags are found in positive reviews, and how many NOUN and VERB tags are found
in negative reviews. Answer the following question: When comparing both, do you see any
differences? Why do you think about the differences? Or lack of them.

Positives had more nouns, while negatives had more verbs. I thikn that makes sense since positive expressions will be more descriptive while, negative expressions will be more active. 

In [14]:
import spacy
import re
nlp = spacy.load("en_core_web_sm")

punctPattern = re.compile("[,!?.:;]")

nounVerbCountByLabel = {}
nounVerbCountByLabel["positive"] = {"noun": 0, "verb": 0}
nounVerbCountByLabel["negative"] = {"noun": 0, "verb": 0}
nounVerbCountByLabel["punctuation"] = 0
count = 0
regexPunctCount = 0
base = "/content/exam1_dataset/TRAINING/"

for filename in allFilenames:
  if filename.endswith(".txt"):
    file = open(base+str(filename), 'r')
    content = file.read()
    # QUESTION 8
    punctMatchCount = re.findall(punctPattern, content)
    regexPunctCount = regexPunctCount + len(punctMatchCount)

    doc = nlp(content)
    for t in doc:
      POS = (spacy.explain(t.pos_)).lower()
      if "noun" in POS:
        if "positive" in filename:
          positiveCount = nounVerbCountByLabel["positive"]
          nounCount = positiveCount["noun"]
          positiveCount["noun"] =nounCount+1
          nounVerbCountByLabel["positive"] = positiveCount
        else:
          negativeCount = nounVerbCountByLabel["negative"]
          nounCount = negativeCount["noun"]
          negativeCount["noun"] =nounCount+1
          nounVerbCountByLabel["negative"] = negativeCount
      elif "verb" in POS:
        if "positive" in filename:
          positiveCount = nounVerbCountByLabel["positive"]
          verbCount = positiveCount["verb"]
          positiveCount["verb"] = verbCount + 1
          nounVerbCountByLabel["positive"] = positiveCount
        else:
          negativeCount = nounVerbCountByLabel["negative"]
          verbCount = negativeCount["verb"]
          negativeCount["verb"] = verbCount +1
          nounVerbCountByLabel["negative"] = negativeCount
      elif "punctuation" in POS:
        punctCount = nounVerbCountByLabel["punctuation"]
        nounVerbCountByLabel["punctuation"] = punctCount + 1

print("Positive")
print("noun", nounVerbCountByLabel["positive"]["noun"])
print("verb", nounVerbCountByLabel["positive"]["verb"])
print("Negative")
print("noun", nounVerbCountByLabel["negative"]["noun"])
print("verb", nounVerbCountByLabel["negative"]["verb"])


Positive
noun 975924
verb 540747
Negative
noun 916579
verb 562823


Question 8) (20 points) Using the results from the PoS process in question 7, count how many
different PUNCT tags are found and their respective counts from the full dataset provided (both
negative and positives together). Using regex, write a set of regular expressions that generate
the same counts from the dataset without using NLTK or Spacy, just regex. Can you get the
same counts? If not, why do you think this is?


I did not receive the same counts, I belive that is because regex didn't take into consideration all possible variations of where a punctuation could be located. Such as I did not account for html characters while spacey did.

In [15]:

print("Regex Punct Total: ", regexPunctCount)
print("spacey punct count: ", nounVerbCountByLabel["punctuation"])

Regex Punct Total:  661014
spacey punct count:  834967
